# **Import&path**

In [25]:
import numpy as np
import pandas as pd
from numpy import errstate,isneginf
from sklearn.preprocessing import normalize
import math
import jieba

# file_path = '/home/nlp/NewsClassify/'
file_path = '/home/nlp/Demo/crawler/'

# **Read file**

In [26]:
column_names = ['type','title','text','time','url']
udn = pd.read_csv(file_path+'udn_demo.tsv',sep = '\t',names=column_names)
cn= pd.read_csv(file_path+'cn_demo.tsv',sep = '\t',names=column_names)
ltn= pd.read_csv(file_path+'ltn_demo.tsv',sep = '\t',names=column_names)


In [27]:
df = pd.concat([udn, cn,ltn], ignore_index=True)
df = df.dropna()
type_dict = {}
## 0：政治 1：生活 2：國際 3：體育 4：娛樂 5：社會 6：財經
type_0 = df[df['type'] == '政治']
type_1 = df[df['type'] == '生活']
type_2 = df[df['type'] == '國際']
type_3 = df[df['type'] == '體育']
type_4 = df[df['type'] == '娛樂']
type_5 = df[df['type'] == '社會']
type_6 = df[df['type'] == '財經']
type_dict['政治'] = 0
type_dict['生活'] = 1
type_dict['國際'] = 2
type_dict['體育'] = 3
type_dict['娛樂'] = 4
type_dict['社會'] = 5
type_dict['財經'] = 6
test = pd.concat([type_0,type_1,type_2,type_3,type_4,type_5,type_6],ignore_index=True)
test = test.drop_duplicates(keep='first',subset = 'text', inplace=False)

In [28]:
# column_names = ['type','title','text']
# df = pd.read_csv(file_path+'all_after_mapping.tsv',sep='\t',names=column_names)
doc_type = test['type'].tolist()
doc = test['text'].tolist()
print(doc_type[0])
print(doc[0])
print(len(doc))

政治
內政部長徐國勇日前氣喘舊疾復發，左肩胛骨扭傷，到醫院急診；徐國勇今受訪表示，「老毛病，我有幾歲，氣喘就有幾年」；總統當天也有打電話關心他了。內政部長徐國勇今出席「空勤總隊黑鷹機隊復編暨廳舍棚廠落成啟用典禮」，健康狀況備受關注，他受訪指出，其實那天是氣喘突然發作，大概就是氣候，還有那幾天空氣不太好，「我這是老毛病了，我有幾歲，氣喘就有幾年」。他說，每天早上大概都要站立很久，可能那天噴藥沒有按照SOP，趕時間，所以突然間就發作。他指出，因為氣喘發作蠻危險，所以當天他就趕快就醫，另一方面是因為以前脊椎受傷，也不知道為什麼氣喘發作又帶動了舊傷，所以整個背部非常痛。他表示，有時候氣喘發作，他也照樣接受質詢，那天業務報告，也撐到下午質詢結束，不過那一天實在是因為背痛肌肉非常痛，才趕去急診，沒有像外界傳的還住院，感謝大家。
8694


In [29]:
print(doc[103])

陽明山擎天崗水牛近日接連暴斃，目前死亡數量已多達30隻，經專家初判可能為「長期營養失衡」導致，但外界仍對水牛的死因議論紛紛。對此，身為獸醫師的台北市議員楊靜宇表示，若營養不良不會造成牛隻密集死亡，應懷疑是否有其他因素。據《ETtoday新聞雲》報導，楊靜宇指出，依目前檢驗報告，牛隻胃部並沒有出血或潰瘍狀況，初步可排除中毒，但對於動保處稱死因是營養不良，持保留態度，因營養不良不會導致密集死亡，或突然大量暴斃，應懷疑是否毒性物質或傳染病影響。牛一天要吃20公斤的草，楊靜宇說，擎天崗山坡改成圍欄後，活動範圍變小，加上可食用的牧草變少，只能靠熱量較低的蘆葦等代替，或改吃其他東西，除本身傳染疾病外，若誤食土壤裡的肉毒桿菌毒素、不該吃的草類、野菇等，也可能導致死亡。此外，楊靜宇20日在臉書指出，擎天崗平時管理單位是陽管處，從今年初開始有些牛隻死亡的處理方式，竟是就地掩埋，但並未通報台北、新北市主管機關動保處，此舉或許造成無法及早發現死亡原因而喪失防疫先機，進而引發大量牛隻死亡，建議台北市、新北市動保處應與陽管處召開會議，依據傳染病防治條例等相關規定，釐清權責及處理SOP機制。楊靜宇表示，人畜共通傳染病的防治非常重要，絕不可掉以輕心，也不能隱瞞疫情造成動物更大規模死亡和民眾疑慮，且世界各國政府的衛生單位、防疫機構，均將此列為重點工作，不要以為其他動物的健康狀況和人類無關，有些疾病是會影響到人類生命安全。


# **TF-IDF**

In [30]:
## 新版
doc_len_list = [] ##doc len
all_doc_data = [] ##all doc split
all_word = {} ##拿來算word_BG的tf值 過濾用
all_word_idf = {}
total_word = 0
## all doc tf 第一次 建好完整的字典
for i in range(len(doc)):
    if(i%1000 ==0):
        print(i)
    update = {}
    d_content = jieba.cut(doc[i])
    d_content_split = []
    for word in d_content:   ##all word
        d_content_split.append(word)
        if(word in all_word):
            all_word[word] +=1
        else:
            all_word[word] = 1
            all_word_idf[word] = 0
        if word not in update:
            all_word_idf[word] += 1
            update[word] = 1
    all_doc_data.append(d_content_split)
print(len(all_word))

0
1000
2000
3000
4000
5000
6000
7000
8000
186383


In [7]:
print(all_word_idf['今天'])

1748


In [31]:
##過濾 tf值 得到新的dict
##過濾透過長度> 2 不含數字 tf>5 出現在95%以下的文章中
new_dict = {} ##總共tf>10或是在query裡的字
# bg_dict = {}
id2word = {}
word2id = {} #建完之後對照用的
id_count = 0
for word in all_word:
    if(all_word_idf[word]>=10):
        word2id[word] = id_count
        id2word[id_count] = word
        id_count += 1
        new_dict[word] = 0
print(len(new_dict))

16605


In [32]:
Doc = len(doc) ##總共幾篇文章
Word = len(new_dict) ##總共幾個字
Query = 1 ##總共幾個query
R = 5 ##relevant 數量
NR = 3 ##最爛幾個Non-relevant
EPOCH = 3 ##Rocchio跑幾次

In [33]:
with errstate(divide='ignore'):
    def TFIDF():
        idf_dict = new_dict.copy()
        tf_idf_matrix = np.zeros([Doc+Query,Word]) ##最終分數
        count = -1
        ## Query TF
#         for q in range(Query):
#             query_content = input()
#             query_tf = new_dict.copy() ##紀錄tf分數
#             update = {}
#             query_content = jieba.cut(query_content)
#             for word in query_content:
#                 if word in new_dict:
#                     query_tf[word] += 1
#                     if word not in update:
#                         idf_dict[word] += 1
#                         update[word] = 0
#             tf_idf_matrix[q] = np.array(list(query_tf.values()))
#         print('Query TF Done!')
        ## Doc TF
        for d in range(Doc):
            doc_content = jieba.cut(doc[d])
            doc_tf = new_dict.copy() ##紀錄tf分數
            update = {}
            for word in doc_content:
                if word in new_dict:
                    doc_tf[word] += 1
                    if word not in update:
                        idf_dict[word] += 1
                        update[word] = 0
            tf_idf_matrix[d+Query] = np.array(list(doc_tf.values()))
        print('Doc TF Done!')
        ## 剛剛算好的idf 做smooth
#         print(idf_dict['intern'])
        for key in idf_dict.keys():
            idf_dict[key] = math.log(1+((Doc)/(idf_dict[key])))
        print('IDF Done!')
        ##算出tf-idf分數
        idf_array = np.array(list(idf_dict.values()))
        for i in range(Doc+Query):
            if i % 1000 == 0:
                print(i)
            score = np.log(tf_idf_matrix[i])
            score += 1
            tf_idf_matrix[i] = np.multiply(idf_array,score)
        print('TF-IDF Done!')
        tf_idf_matrix[isneginf(tf_idf_matrix)]=0
        tf_idf_matrix = normalize(tf_idf_matrix,norm ='l2')
        print('Norm Done!')
        return tf_idf_matrix

In [34]:
tf_idf_matrix = TFIDF()
print(tf_idf_matrix[0])

Doc TF Done!
IDF Done!
0
1000


/home/nlp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log


2000
3000
4000
5000
6000
7000
8000
TF-IDF Done!
Norm Done!
[0. 0. 0. ... 0. 0. 0.]


In [35]:
np.save('./numpy_IR/demo_tfidf',tf_idf_matrix)

In [13]:
tf_idf_matrix = np.load('./numpy_IR/demo_tfidf.npy')

In [88]:
idf_dict = new_dict.copy()
for q in range(Query):
    query_content = input()
    query_tf = new_dict.copy() ##紀錄tf分數
    update = {}
    query_content = jieba.cut(query_content)
    for word in query_content:
        if word in new_dict:
            query_tf[word] += 1
            if word not in update:
                idf_dict[word] += 1
                update[word] = 0
    tf_idf_matrix[q] = np.array(list(query_tf.values()))

蛋糕


In [89]:
ans_list = []
ans_index_list = []
for i in range(Query): ##總共幾個query
    query_ans_list = []   ##排序輸出結果
    score_list = np.matmul(tf_idf_matrix[Query:],tf_idf_matrix[i]) 
    result_score = sorted(range(Doc),reverse = True,key = lambda k :score_list[k])
    for j in range(Doc):
        query_ans_list.append(doc[result_score[j]])
    print((query_ans_list[0]))
    for k in range(100):
        ans_list.append(query_ans_list[k])
        ans_index_list.append(result_score[k]) ##relevant index

黑船回來了！來自日本東京的甜點「黑船Quolofune」，將於1月3日起於台北101展開為期2個月的快閃店活動，除了招牌的長崎蛋糕、長崎蛋糕脆條RASQ等甜品外，還為春節特別推出「春節刻印蛋糕」，以及特選禮盒。1月3日至1月10日期間，還可享買「檸檬蛋糕送銅鑼燒」的優惠活動。源自東京自由之丘的黑船，創立自2006年，並於2017年首度登台，陸續開設台北忠孝SOGO店、高雄漢神店。但在今年10月起，兩間門市陸續熄燈，銷售模式也暫轉以日本展、快閃、電商平台等型態。此次黑船即以快閃店型態重回市場。快閃店所供應的「長崎蛋糕」乃是黑船的主打商品，遵循古法配方製作，不加蜂蜜仍有蜂蜜般的香氣，底鋪也藏有雙目糖的粗粒結晶，每條600元，另外還有抹茶、黑糖等口味的單片蛋糕。另外店內也有長崎蛋糕脆條RASQ、波浪形華夫餅乾、檸檬蛋糕等產品。因應春節送禮需求所推出的「春節刻印蛋糕」，以檸檬蛋糕為基礎，搭配台灣龍眼蜜、日本糖漬檸檬丁而成，並印有「迎春」圖樣，每條400元。「黑船特選禮盒」則是以銅鑼燒為主，每盒915元。1月20日前預定，可享9折優惠。


In [90]:
print(ans_index_list)

[1386, 4717, 2004, 1179, 1449, 3131, 2192, 2700, 2014, 1247, 4776, 1897, 1768, 1199, 705, 2087, 6789, 6846, 1551, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]


In [92]:
print(len(ans_list))
print(ans_list[0])
print(ans_index_list[0])
print(doc[99])

100
黑船回來了！來自日本東京的甜點「黑船Quolofune」，將於1月3日起於台北101展開為期2個月的快閃店活動，除了招牌的長崎蛋糕、長崎蛋糕脆條RASQ等甜品外，還為春節特別推出「春節刻印蛋糕」，以及特選禮盒。1月3日至1月10日期間，還可享買「檸檬蛋糕送銅鑼燒」的優惠活動。源自東京自由之丘的黑船，創立自2006年，並於2017年首度登台，陸續開設台北忠孝SOGO店、高雄漢神店。但在今年10月起，兩間門市陸續熄燈，銷售模式也暫轉以日本展、快閃、電商平台等型態。此次黑船即以快閃店型態重回市場。快閃店所供應的「長崎蛋糕」乃是黑船的主打商品，遵循古法配方製作，不加蜂蜜仍有蜂蜜般的香氣，底鋪也藏有雙目糖的粗粒結晶，每條600元，另外還有抹茶、黑糖等口味的單片蛋糕。另外店內也有長崎蛋糕脆條RASQ、波浪形華夫餅乾、檸檬蛋糕等產品。因應春節送禮需求所推出的「春節刻印蛋糕」，以檸檬蛋糕為基礎，搭配台灣龍眼蜜、日本糖漬檸檬丁而成，並印有「迎春」圖樣，每條400元。「黑船特選禮盒」則是以銅鑼燒為主，每盒915元。1月20日前預定，可享9折優惠。
1386
爭議論多年的「萊豬進口問題」今年點燃引信。8月28日蔡英文宣布開放含萊克多巴胺的美國豬肉進口，此議題立即成為朝野攻防重點，隨後爆發連串政黨、民間抗爭。今（24）日立法院針對萊豬行政命令進行表決大戰，萊豬叩關進入倒數之際，《中時新聞網》帶您回顧這場萊豬大戰始末，目證歷史記錄。8月28日蔡英文總統宣布終結14年禁止瘦肉精豬肉進口後，引發各界譁然。隨後，主管單位衛褔部公布萊克多巴胺殘留容許值，豬肝、豬腎0.04ppm，但台灣並未進口，其他可食用部位定為0.01ppm。11月22日勞工運動團體發起「秋鬥」抗議遊行，5萬民眾上街對萊豬政策發出怒吼，堪稱是歷年來活動人數最盛大的一次。隨後蘇貞昌就萊豬開放一事，到立法院進行施政報告，國民黨發動12次杯葛。11月27日蘇貞昌第13度赴報告，國民黨無預警潑灑豬內臟，引爆國會衝突。12月12日，沈智慧在立法院門口絕食，抗議來豬進口，16上午11時許絕食進入第97小時，體力不支，經醫師評估後送醫治療，由沈智慧發起的野薑花運動也不斷擴大。在此同時，政府「查水表」事件一波接一波。先是信功肉品指控蘇貞昌故意誤導，強調支持的是台灣豬肉外銷、反對萊豬，沒多久就有消防人員上門安檢，對此行政院澄清是業者自行提出

In [93]:
print(doc_type[6198])
print(type_dict['國際'])

社會
2


In [63]:
for i in ans_index_list:
    print(doc_type[i])

國際
國際
國際
財經
財經
財經
政治
政治
國際
國際
政治
國際
國際
國際
國際
國際
國際
國際
國際
國際
國際
政治
政治
國際
國際
國際
國際
國際
國際
國際
國際
政治
財經
財經
國際
國際
國際
國際
財經
國際
國際
國際
財經
財經
生活
國際
國際
國際
國際
國際
國際
生活
生活
國際
國際
財經
國際
國際
國際
國際
財經
國際
國際
國際
國際
國際
國際
國際
國際
國際
國際
國際
國際
財經
財經
國際
國際
國際
國際
國際
生活
生活
生活
政治
國際
國際
國際
國際
政治
政治
生活
生活
生活
生活
國際
財經
財經
生活
生活
政治


In [94]:
result_list = [] ##分類過後的輸出
used_idx = [] ## already output
used_type =[0,0,0,0,0,0,0]
for i in range(20):  ##找到七篇就停止
    count = 0
    for check in range(7):
        if used_type[check] != 0:
            count += 1
    if count == 7: ## all type find
        break
    if used_type[type_dict[doc_type[i]]] == 0:
        result_list.append(ans_list[i])
        used_type[type_dict[doc_type[i]]] += 1  ## used
        used_idx.append(i) ## used idx
print(len(result_list))
for i in range(100): ##再找13篇
    if i not in used_idx:
        result_list.append(ans_list[i])
    count = len(result_list)
    if count == 20:
        break
# result

1


In [95]:
print(result_list[1])

林心如、霍建華從好友升格成戀人，小倆口在2016年5月公布戀情，並在同年7月選在峇里島舉行婚禮，小倆口接著在隔年喜迎女兒「小海豚」，幸福升格為一家三口，而昨（26日）是霍建華的41歲生日，林心如也貼心為老公準備法拉利造型蛋糕，還釣出侯佩岑留言讚：「格格真是好有心世紀好太太」。霍建華昨（26日）迎接41歲生日，老婆林心如則貼心為霍建華準備紅色法拉利的造型蛋糕，儘管林心如只有簡單發文：「Happybirthday」、並貼上4個愛心貼圖，不過蛋糕則是有滿滿的心意。只見蛋糕旁寫著霍建華的英文名字「Wallace」，其中還有「DaddyHappyBirthday」字樣，看來是以女兒小海豚的口吻送給爸爸的，而照片也釣出侯佩岑留言：「皇上生日快樂！」而侯佩岑還不忘大讚林心如：「格格真是好有心世紀好太太」。霍建華雖然對生活低調，不過他過去就曾被媒體拍到，駕駛法拉利、賓士車上路，其中還有幾輛是全球限量款，市值超過3千萬元，也讓外界驚嘆，原來霍建華是愛車一族，以及「皇上」的財力如此雄厚。


# **弄出結果**

In [96]:
with open('./result_VSM_20.txt','w') as f:
    for i in range(Query): 
        for j in range(len(result_list)):
            f.write(str(j+1)+'\n')
            f.write(result_list[j])
            f.write('\n')
        f.write('\n')
    f.close()


In [22]:
## BM25跑出來的relevant
bm = np.load('./numpy/BM_relevant.npy')

[12115. 24686. 29949. 21615. 25528.]


In [23]:
## plsa 跑出來的relevant
plsa = np.load('./numpy/PLSA_relevant.npy')

[24686 12115 18880 25247 29949]


In [58]:
## find VSM relevant
query_relevant =np.zeros([Query,R])
query_N_relevant = np.zeros([Query,NR])
for q in range(Query):
    query_relevant[q] = ans_index_list[q][:R]
    query_N_relevant[q] = ans_index_list[q][-NR:]
print(query_relevant[0])
print(query_N_relevant[0])

[20382. 29414. 35410. 12636.  9135.]
[35543. 35544. 35545.]


In [42]:
def output_ans(ans_list,epoch):
    with open('./Rocchio_'+str(epoch)+'.txt','w') as f:
        for i in range(Query): 
            for j in range(len(ans_list)):
                f.write(str(j+1)+'\n')
                f.write(ans_list[j])
                f.write('\n')
            f.write('\n')
        f.close()


In [59]:
for e in range(EPOCH):
    print('Epoch:',str(e+1))
    a = 0.98 ##原始query
    b = 0.75 ##relevant doc
    c = 0.15  ##non-relevant doc
    all_ans_list = []
    for i in range(Query):
        query_ans_list = []
        score_list = [] ##每篇query的分數
        relevant = np.zeros([Word])
        N_relevant = np.zeros([Word])
        new_query = np.zeros([Word])
        new_query += a * tf_idf_matrix[i]
        for j in range(R):
            relevant += tf_idf_matrix[Query+int(query_relevant[i][j])]
#         if(e<3): ##只讓BM & PLSA參與前三次更新
#             for j in range(R): ## BM
#                 relevant += tf_idf_matrix[Query+int(bm[i][j])]
#             for j in range(R): ## BM
#                 relevant += tf_idf_matrix[Query+int(plsa[i][j])]
        for j in range(NR):
            N_relevant +=tf_idf_matrix[Query+int(query_N_relevant[i][j])]
#         if(e<3): ##只讓BM & PLSA參與前三次更新
#             relevant /= (R*3)
#         else:
        relevant /= R
        N_relevant /= NR
        new_query += b * relevant
        new_query -= c * N_relevant
        tf_idf_matrix[i] = new_query
        for d in range(Doc):
            score_list.append(np.matmul(tf_idf_matrix[Query+d],tf_idf_matrix[i]))
        result_score = sorted(range(Doc),reverse = True,key = lambda k :score_list[k])
        if(e % 1 == 0):
            for j in range(Doc):
                query_ans_list.append(doc[result_score[j]])
            for k in range(10):
                all_ans_list.append(query_ans_list[k])
        query_relevant[i] = result_score[:R]
        query_N_relevant[i] = result_score[-NR:]
    if(e % 1 == 0):
        output_ans(all_ans_list,e)

Epoch: 1
Epoch: 2
Epoch: 3


In [97]:
import flask
flask.__version__

'1.1.2'